### RespAid

In [26]:
import tkinter as tk
from tkinter import ttk, messagebox, filedialog

# Define the knowledge base (same as in the original code)
knowledge_base = [
    {"conditions": {"temperature_high", "cough dry", "body_aches"}, 
     "diagnosis": "Influenza",
     "recommendation": "Rest, stay hydrated, and take antiviral medications if prescribed."},
    {"conditions": {"temperature_moderate", "cough dry", "loss_of_taste_smell"}, 
     "diagnosis": "COVID-19",
     "recommendation": "Isolate yourself, consult a doctor, and follow the health guidelines for COVID-19."},
    {"conditions": {"temperature_high", "cough productive", "chest_pain" }, 
     "diagnosis": "Pneumonia",
     "recommendation": "Seek immediate medical attention, rest, and take antibiotics if prescribed."},
    {"conditions": {"cough productive", "mucus_production", "sore_throat"}, 
     "diagnosis": "Bronchitis",
     "recommendation": "Rest, drink fluids, and use expectorants to ease mucus production."},
    {"conditions": {"wheezing", "shortness_of_breath", "chest_tightness"}, 
     "diagnosis": "Asthma",
     "recommendation": "Use inhalers as prescribed and avoid triggers that may cause an asthma attack."},
    {"conditions": {"cough productive", "shortness_of_breath", "history_of_smoking"}, 
     "diagnosis": "COPD",
     "recommendation": "Quit smoking, use inhalers, and consult a pulmonologist for further management."},
    {"conditions": {"cough productive", "weight_loss", "night_sweats"}, 
     "diagnosis": "Tuberculosis",
     "recommendation": "Seek immediate medical attention for tuberculosis testing and treatment."}
]

all_symptoms = [
    "temperature_low", "temperature_moderate", "temperature_high", "cough dry", "cough productive",
    "body_aches", "sore_throat", "fatigue", "nasal_congestion", "shortness_of_breath",
    "persistent_fever", "loss_of_taste_smell", "mucus_production", "chest_pain",
    "chills", "wheezing", "chest_tightness", "history_of_smoking", "weight_loss",
    "night_sweats", "blood_in_sputum"
]

def infer(symptoms):
    for rule in knowledge_base:
        if all(symptom in symptoms for symptom in rule["conditions"]) and len(symptoms) == len(rule["conditions"]):
            return rule["diagnosis"], rule["recommendation"]
    return "No diagnosis found", "Please consult a doctor for a professional evaluation."

class ExpertSystemApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Medical Expert System ")
        self.root.geometry("800x600")

        self.base_font_size = 14  
        self.setup_home_page()

    def setup_home_page(self):
        # Welcome Screen
        self.home_frame = tk.Frame(self.root, bg="white")
        self.home_frame.pack(fill="both", expand=True)

        self.welcome_label = tk.Label(self.home_frame, text="", font=("Arial", 28), bg="white", fg="#27AE60")
        self.welcome_label.pack(pady=50)

        start_button = tk.Button(
            self.home_frame, text="Start Diagnosis", command=self.setup_ui,
            bg="#2ECC71", fg="white", font=("Arial", 17)
        )
        start_button.pack(pady=20)

        # Start typewriter animation
        self.animate_text("Welcome to RespAid")

    def animate_text(self, text, idx=0):
        if idx < len(text):
            current_text = text[:idx + 1]
            self.welcome_label.config(text=current_text)
            self.root.after(50, self.animate_text, text, idx + 1)

    def setup_ui(self):
        # Remove home screen
        self.home_frame.pack_forget()

        # UI Setup (Main Screen)
        self.main_frame = tk.Frame(self.root, bg="white")
        self.main_frame.pack(fill="both", expand=True)

        # Search bar
        search_frame = tk.Frame(self.main_frame, bg="white")
        search_frame.pack(fill="x", padx=10, pady=5)
        tk.Label(search_frame, text="Search Symptoms:", font=("Arial", 18), bg="white").pack(side="left")  # Larger font size
        self.search_var = tk.StringVar()
        self.search_var.trace("w", self.filter_symptoms)
        tk.Entry(search_frame, textvariable=self.search_var, font=("Arial", 16), width=30).pack(side="left", padx=5)  # Larger font size

        # Symptoms list
        self.symptoms_frame = tk.Frame(self.main_frame, bg="white")
        self.symptoms_frame.pack(fill="both", expand=True, padx=10, pady=5)

        self.symptoms_canvas = tk.Canvas(self.symptoms_frame, bg="white")
        self.symptoms_canvas.pack(side="left", fill="both", expand=True)

        self.scrollbar = ttk.Scrollbar(self.symptoms_frame, orient="vertical", command=self.symptoms_canvas.yview)
        self.scrollbar.pack(side="right", fill="y")

        self.symptoms_list_frame = tk.Frame(self.symptoms_canvas, bg="white")
        self.symptoms_canvas.create_window((0, 0), window=self.symptoms_list_frame, anchor="nw")
        self.symptoms_canvas.configure(yscrollcommand=self.scrollbar.set)

        # Diagnose button
        self.diagnose_button = tk.Button(self.main_frame, text="Diagnose", command=self.diagnose, bg="#00695C", fg="white", font=("Arial", 16))  # Larger font size
        self.diagnose_button.pack(pady=10)

        # Diagnosis and Recommendation
        self.result_label = tk.Label(self.main_frame, text="", wraplength=750, justify="center", bg="white", font=("Arial", 16))  # Larger font size
        self.result_label.pack(pady=10)
        self.recommendation_label = tk.Label(self.main_frame, text="", wraplength=750, justify="center", bg="white", font=("Arial", 16))  # Larger font size
        self.recommendation_label.pack(pady=10)

        # Export Button
        self.export_button = tk.Button(self.main_frame, text="Export Results", command=self.export_results, bg="green", fg="white", font=("Arial", 16))  # Larger font size
        self.export_button.pack(pady=10)

        # Clean Form Button
        self.clean_button = tk.Button(self.main_frame, text="Clear Form", command=self.clear_form, bg="#002D62", fg="white", font=("Arial", 16))  # Larger font size
        self.clean_button.pack(pady=10)

        # Load symptoms for the diagnosis screen
        self.load_symptoms()

    def load_symptoms(self):
        self.symptom_vars = {}
        self.checkbox_widgets = {}  # Keep track of checkbox widgets for dynamic hiding

        # Create the symptoms checkboxes
        for symptom in all_symptoms:
            var = tk.BooleanVar()
            self.symptom_vars[symptom] = var
            chk = tk.Checkbutton(
                self.symptoms_list_frame,
                text=symptom.capitalize().replace("_", " "),
                variable=var,
                font=("Arial", 12),  # Increased font size for checkboxes
                bg="white",
                name=f"checkbutton{symptom}")
            chk.pack(anchor="w")
            self.checkbox_widgets[symptom] = chk  # Store reference to checkbox

        self.symptoms_list_frame.update_idletasks()
        self.symptoms_canvas.config(scrollregion=self.symptoms_canvas.bbox("all"))

    def filter_symptoms(self, *args):
        query = self.search_var.get().lower()
        for symptom, widget in self.checkbox_widgets.items():
            # Hide symptom if it doesn't match the query
            if query in symptom.lower():
                widget.pack(anchor="w")  # Show it
            else:
                widget.pack_forget()  # Hide it

    def diagnose(self):
        # Get selected symptoms
        selected_symptoms = [symptom for symptom, var in self.symptom_vars.items() if var.get()]

        if not selected_symptoms:
            messagebox.showerror("Input Error", "Please select at least one symptom.")
            return

        diagnosis, recommendation = infer(selected_symptoms)
        severity_color = "green" if diagnosis != "No diagnosis found" else "red"
        self.result_label.config(text=f"Diagnosis: {diagnosis}", fg=severity_color)
        self.recommendation_label.config(text=f"Recommendation: {recommendation}")

    def clear_form(self):
        for var in self.symptom_vars.values():
            var.set(False)
        self.result_label.config(text="")
        self.recommendation_label.config(text="")

    def export_results(self):
        result_text = f"{self.result_label.cget('text')}\n\n{self.recommendation_label.cget('text')}"
        file_path = filedialog.asksaveasfilename(defaultextension=".txt", filetypes=[("Text Files", "*.txt")])
        if file_path:
            with open(file_path, "w") as file:
                file.write(result_text)
            messagebox.showinfo("Export Successful", f"Results exported to {file_path}")

    def adjust_font_sizes(self, event=None):
        width, height = self.root.winfo_width(), self.root.winfo_height()
        scale_factor = min(width / 800, height / 600)
        new_font_size = int(self.base_font_size * scale_factor)

        # Update font sizes
        self.diagnose_button.config(font=("Arial", new_font_size))
        self.export_button.config(font=("Arial", new_font_size))
        self.clean_button.config(font=("Arial", new_font_size))
        self.result_label.config(font=("Arial", new_font_size + 2))
        self.recommendation_label.config(font=("Arial", new_font_size))

        for widget in self.symptoms_list_frame.winfo_children():
            widget.config(font=("Arial", new_font_size))

if __name__ == "__main__":
    root = tk.Tk()
    app = ExpertSystemApp(root)
    root.mainloop()
